<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AI_Text_Adventur_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import IPython.display as display
from google.colab import userdata
from google.colab import output
import requests
import json
import time # For exponential backoff
import asyncio # Import asyncio
import nest_asyncio # Import nest_asyncio
import sqlite3 # New import for the database

# Apply nest_asyncio to allow asyncio to run inside a running event loop
nest_asyncio.apply()

# Google Colab Specific API Key Setup
# This part assumes you are running in Google Colab and have stored your API key
try:
    GOOGLE_API_KEY = userdata.get('GEMINI')
    print("Google Generative AI configured successfully using Colab Secrets.")
except ImportError:
    print("Not in Google Colab, or 'userdata' module not found. Please set GOOGLE_API_KEY manually.")
    GOOGLE_API_KEY = None # Fallback for local execution

LLM_MODEL_NAME = "gemini-2.5-flash-preview-05-20" # Explicitly use the preferred model

# --- Game Configuration ---
TOTAL_GAME_TURNS = 5 # Define the total number of turns as a variable
POINTS_PER_CORRECT_ANSWER = 100 / TOTAL_GAME_TURNS # Calculate points per correct answer based on total turns

# Global Game State
game_chat_history = []
current_story_description = "Welcome, young AI explorer! Click 'Start New AI Adventure' to begin your journey into the world of smart machines! 🤖"
current_choices = []
game_over_status = False
loading_message = "Exploring the digital frontier... Please wait. "
error_message = ""
info_message = ""
player_score = 0 # Global variable for the player's score
turn_counter = 0 # Global variable for tracking turns

# --- Database Functions ---
# Note: Database functions are kept for potential future use or compatibility,
# but high scores are now handled via Google Drive in this version.
def setup_database():
    """
    Creates the SQLite database and a table for high scores if they don't exist.
    (Note: High scores are currently saved to Google Drive).
    """
    conn = sqlite3.connect('ai_explorer.db') # Changed database name
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS scores (
            player_name TEXT NOT NULL,
            score INTEGER NOT NULL
        );
    """)
    conn.commit()
    conn.close()
    print("Database setup complete (using SQLite, though high scores go to Drive).")


def save_high_score(player_name, score):
    """
    Saves a player's score to the SQLite database (Note: Not used for high scores persistence in this version).
    """
    try:
        conn = sqlite3.connect('ai_explorer.db') # Changed database name
        cursor = conn.cursor()
        cursor.execute("INSERT INTO scores (player_name, score) VALUES (?, ?)", (player_name, score))
        conn.commit()
        conn.close()
        print(f"Score for {player_name} saved to SQLite (High scores saved to Drive).")
    except sqlite3.Error as e:
        print(f"Database error: {e}")

def get_high_scores_from_db():
    """
    Retrieves the top 5 high scores from the SQLite database (Note: Not used for high scores persistence in this version).
    """
    scores = []
    try:
        conn = sqlite3.connect('ai_explorer.db') # Changed database name
        cursor = conn.cursor()
        cursor.execute("SELECT player_name, score FROM scores ORDER BY score DESC LIMIT 5")
        scores = cursor.fetchall()
        conn.close()
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    return scores

# --- Google Drive Persistence Functions ---
# Ensure Google Drive is mounted if using persistence
try:
    from google.colab import drive
    import os
    import pandas as pd

    # Mount Google Drive (this will prompt for authorization the first time)
    # drive.mount('/content/drive') # Mounting happens once, usually in a separate cell

    # Define the path for the high scores file in Google Drive
    # Create a directory for the game data if it doesn't exist
    game_data_dir = '/content/drive/My Drive/AIExplorerGameData'
    os.makedirs(game_data_dir, exist_ok=True)
    HIGH_SCORES_FILE = os.path.join(game_data_dir, 'high_scores.csv')

    def load_high_scores_from_drive():
        """
        Loads high scores from a CSV file in Google Drive.
        Returns a list of (player_name, score) tuples, sorted by score descending.
        Handles file not found or errors gracefully.
        """
        scores = []
        if os.path.exists(HIGH_SCORES_FILE):
            try:
                # Read the CSV file into a pandas DataFrame
                df_scores = pd.read_csv(HIGH_SCORES_FILE)
                # Ensure required columns exist and handle potential parsing errors
                if 'player_name' in df_scores.columns and 'score' in df_scores.columns:
                     # Convert score column to numeric, coercing errors to NaN
                    df_scores['score'] = pd.to_numeric(df_scores['score'], errors='coerce')
                    # Drop rows with NaN scores if any occurred during coercion
                    df_scores.dropna(subset=['score'], inplace=True)
                    # Convert score to integer
                    df_scores['score'] = df_scores['score'].astype(int)
                    # Sort by score descending
                    df_scores = df_scores.sort_values(by='score', ascending=False)
                    # Convert DataFrame to list of tuples
                    scores = list(df_scores[['player_name', 'score']].itertuples(index=False, name=None))
                else:
                    print(f"Warning: High scores file '{HIGH_SCORES_FILE}' is missing required columns. Starting fresh.")
            except pd.errors.EmptyDataError:
                 print(f"Info: High scores file '{HIGH_SCORES_FILE}' is empty. Starting fresh.")
            except Exception as e:
                print(f"Error loading high scores from Drive: {e}. Starting fresh.")
        else:
            print(f"Info: High scores file '{HIGH_SCORES_FILE}' not found. Starting fresh.")
        return scores

    def save_high_scores_to_drive(scores_list):
        """
        Saves a list of (player_name, score) tuples to a CSV file in Google Drive.
        Handles potential errors during saving.
        """
        try:
            # Create a pandas DataFrame from the list of scores
            df_scores = pd.DataFrame(scores_list, columns=['player_name', 'score'])
            # Save the DataFrame to CSV
            df_scores.to_csv(HIGH_SCORES_FILE, index=False)
            print(f"High scores saved to {HIGH_SCORES_FILE}")
        except Exception as e:
            print(f"Error saving high scores to Drive: {e}")

    # Use the Drive functions for the main game logic
    get_high_scores = load_high_scores_from_drive

except ImportError:
    print("Google Drive module not available. Using in-memory storage for high scores.")
    # Fallback to in-memory storage if Drive is not available
    # Note: Scores will not persist across sessions in this fallback.
    _in_memory_high_scores = []

    def load_high_scores_from_drive():
        global _in_memory_high_scores
        # Sort in-memory scores
        _in_memory_high_scores.sort(key=lambda item: item[1], reverse=True)
        return _in_memory_high_scores

    def save_high_scores_to_drive(scores_list):
        global _in_memory_high_scores
        _in_memory_high_scores = scores_list
        print("Scores saved to in-memory storage.")

    # Use the in-memory fallback functions
    get_high_scores = load_high_scores_from_drive # Alias for compatibility


# LLM API Call Function (Kept the same as it's already robust)
async def call_llm(prompt_text, max_retries=5, initial_delay=1.0):
    """
    Calls the Gemini LLM API to get simulation parameters.
    Implements exponential backoff for retries.
    """
    global game_chat_history, loading_message, error_message, info_message

    # Clear previous messages
    error_message = ""
    info_message = ""

    # Check if API key is available
    if not GOOGLE_API_KEY:
        error_message = "Google API Key not available. Cannot call LLM."
        print(f"ERROR: {error_message}")
        update_game_ui(is_loading=False) # Ensure UI updates with error
        return None

    api_url = f"https://generativelanguage.googleapis.com/v1beta/models/{LLM_MODEL_NAME}:generateContent?key={GOOGLE_API_KEY}"
    headers = {"Content-Type": "application/json"}

    # Add the prompt to history regardless of whether it's an initial prompt or an action.
    # The prompt_text is always the user's "turn" in this context.
    game_chat_history.append({"role": "user", "parts": [{"text": prompt_text}]})

    payload = {
        "contents": game_chat_history,
        "generationConfig": {
            "responseMimeType": "application/json",
            "responseSchema": {
                "type": "OBJECT",
                "properties": {
                    "description": {"type": "STRING", "description": "The current story description of the AI adventure."},
                    "choices": {
                        "type": "ARRAY",
                        "items": {"type": "STRING"},
                        "description": "List of possible actions/choices for the player in the AI world."
                    },
                    "gameover": {"type": "BOOLEAN", "description": "True if the game is over (e.g., player learned a concept, or failed an AI challenge)."},
                    "score_change": {"type": "INTEGER", "description": "The number of points gained or lost based on the player's last action."},
                    "concept_explained": {"type": "STRING", "description": "The AI concept explained or a brief summary of the key learning from the last action."},
                    "correct_answer_feedback": {"type": "STRING", "description": "Feedback on the correct answer if the player made a mistake."}
                },
                "required": ["description", "choices", "gameover", "score_change", "concept_explained", "correct_answer_feedback"]
            }
        }
    }

    retries = 0
    delay = initial_delay

    while retries < max_retries:
        try:
            # Indicate loading in the UI *before* the actual request
            update_game_ui(is_loading=True)
            response = requests.post(api_url, headers=headers, data=json.dumps(payload))
            response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
            result = response.json()

            if result and result.get("candidates") and result["candidates"][0]["content"].get("parts"):
                llm_data_str = result["candidates"][0]["content"]["parts"][0].get("text")
                if not llm_data_str:
                    error_message = "LLM response missing expected 'text' content."
                    print(f"ERROR: {error_message}")
                    return None

                # Ensure it's a string that can be parsed as JSON
                if isinstance(llm_data_str, dict): # Handle cases where LLM might return dict directly
                    parsed_json = llm_data_str
                else:
                    parsed_json = json.loads(llm_data_str)

                # Add LLM's response to chat history
                game_chat_history.append({"role": "model", "parts": [{"text": llm_data_str}]})
                return parsed_json
            else:
                error_message = f"LLM response missing expected content structure. Result: {result}"
                print(f"ERROR: {error_message}")
                if "blocked_reason" in str(result): # Example of checking for safety blocking
                    error_message = "LLM response was blocked, potential safety issue."
                    print(f"ERROR: {error_message}")
                return None

        except requests.exceptions.RequestException as e:
            error_message = f"API request failed (Attempt {retries + 1}/{max_retries}): {e}"
            print(f"ERROR: {error_message}")
            if response is not None:
                print(f"Response status: {response.status_code}, Body: {response.text[:500]}...")
            retries += 1
            if retries < max_retries:
                await asyncio.sleep(delay)
                delay *= 2 # Exponential backoff
        except json.JSONDecodeError as e:
            error_message = f"Error parsing JSON response from LLM: {e}. Raw response: {response.text[:500]}..."
            print(f"ERROR: {error_message}")
            return None
        except Exception as e:
            error_message = f"An unexpected error occurred during API call: {e}"
            print(f"ERROR: {error_message}")
            return None

# Game Logic Functions (Python Callbacks)

def initial_game_prompt():
    """
    Generates the initial prompt for the LLM to set up the AI explorer game scenario.
    This prompt introduces a child protagonist to the world of AI through a friendly AI assistant or device,
    introduces a basic AI concept, and asks the LLM to provide the initial story description,
    choices, and game state.
    """
    return """You are the game master for a text adventure designed to teach basic Artificial Intelligence (AI) concepts to children aged 8-12.
    Generate an initial scenario where a child protagonist discovers a friendly AI assistant or device that introduces them to the world of AI.
    The language should be simple, engaging, and suitable for children.
    Introduce a very basic AI concept or capability (e.g., AI recognizing objects, following instructions, learning simple patterns).
    Provide a 'description' (a few sentences, 100-200 words) and a 'choices' array (2-4 child-friendly options related to interacting with or exploring AI).
    Set 'gameover' to false. Also provide a 'score_change' of 0 and an empty 'concept_explained' for the start of the game. Also provide an empty 'correct_answer_feedback' field.

    Example: {"description": "You're playing in your treehouse when you find a dusty old tablet. You press a button, and a friendly face appears on the screen! 'Hello!' it chirps. 'I'm your AI friend, Sparky! I can help you learn about how computers can think and learn. What do you want to explore first?'", "choices": ["Ask Sparky what AI is.", "Ask Sparky to recognize your toys.", "Close the tablet, it's weird."], "gameover": false, "score_change": 0, "concept_explained": "", "correct_answer_feedback": ""}
    """

async def start_new_game():
    """
    Resets the game state and initiates a new AI adventure scenario.
    Loads high scores from Google Drive or in-memory storage on start.
    """
    global game_chat_history, current_story_description, current_choices, game_over_status, error_message, info_message, player_score, turn_counter, TOTAL_GAME_TURNS, POINTS_PER_CORRECT_ANSWER
    game_chat_history = [] # Reset chat history
    game_over_status = False
    error_message = ""
    info_message = ""
    current_story_description = "Welcome, young AI explorer! Click 'Start New AI Adventure' to begin your journey into the world of smart machines! 🤖" # Reset to initial welcome
    current_choices = []
    player_score = 0 # Reset the score for a new game
    turn_counter = 0 # Reset the turn counter
    POINTS_PER_CORRECT_ANSWER = 100 / TOTAL_GAME_TURNS # Recalculate points per correct answer for the new game

    # High scores are now handled via Google Drive or in-memory storage
    # setup_database() # No longer needed for high scores persistence via SQLite

    update_game_ui(is_loading=True) # Show loading while fetching

    scenario = await call_llm(initial_game_prompt())

    if scenario:
        current_story_description = scenario['description']
        current_choices = scenario['choices']
        game_over_status = scenario['gameover']
        # No score change on initial load
    else:
        current_story_description = "Failed to start new game. Please check for API key errors or try again."
        current_choices = []
        game_over_status = True # End game on error
        error_message = "Error: Could not retrieve initial scenario."
    update_game_ui(is_loading=False)

async def handle_player_action(player_action):
    """
    Processes the player's action and updates the game story based on LLM response,
    integrating a wider range of AI concepts with increasing complexity over turns,
    and handling scoring.
    """
    global current_story_description, current_choices, game_over_status, error_message, info_message, player_score, turn_counter, TOTAL_GAME_TURNS, POINTS_PER_CORRECT_ANSWER

    if not player_action.strip():
        error_message = "Please enter your action or click a choice button."
        update_game_ui(is_loading=False) # Ensure loading is off and error is displayed
        return

    # Increment the turn counter with each player action
    turn_counter += 1

    # Append player's action to history for immediate feedback, before LLM call
    # Only append if not the final turn description
    if turn_counter <= TOTAL_GAME_TURNS: # Check if it's not the turn *after* the final turn
        current_story_description += f'\n\nYou chose: "{player_action}"'
    update_game_ui(is_loading=True) # Show loading while fetching

    # This prompt guides the LLM to advance the AI-themed story,
    # introduce and explain AI concepts relevant to the player's choice,
    # determine the correct choice for learning, assign scores (20 for correct, 0 otherwise),
    # provide feedback for incorrect answers, and generate the next turn's game state.
    # We explicitly tell the LLM not to set gameover to true, as we control it by turn counter.
    # Instructions are added to introduce different concepts with increasing complexity per turn.
    game_prompt = f"""
    You are the game master for a text adventure designed to teach basic Artificial Intelligence (AI) concepts to children aged 8-12.
    The player chose: "{player_action}".
    This is turn number {turn_counter} out of {TOTAL_GAME_TURNS}.
    Given the previous story and the player's choice, advance the narrative, keeping the language simple and engaging, focusing on AI.

    For this turn ({turn_counter}), introduce or reinforce a *different* specific AI concept relevant to the choice than previously covered. Try to build upon earlier concepts if logical.
    As the turns progress from 1 to {TOTAL_GAME_TURNS}, slightly increase the complexity or add a bit more detail to the AI concept explanation.
    Choose from a range of child-friendly AI concepts such as:
    - What is AI?
    - Learning from data (Machine Learning)
    - Following rules (Algorithms)
    - Recognizing pictures (Computer Vision)
    - Understanding words (Natural Language Processing)
    - Robots and Automation
    - Training AI (showing it examples)
    - How AI makes decisions
    - AI helping people
    - AI in games

    Out of the 3-4 choices you provide, determine which one is the single most correct or best choice that helps the player learn the concept or explore the AI world positively.
    If the player chose the correct option, provide a 'score_change' of {int(POINTS_PER_CORRECT_ANSWER)}.
    For any other choice, provide a 'score_change' of 0.

    If the player gets 0 points, provide a brief 'correct_answer_feedback' that gives a hint about the right answer or explains why the chosen option was not the best for learning about AI in this situation.
    If the player gets {int(POINTS_PER_CORRECT_ANSWER)} points, the 'correct_answer_feedback' should be an empty string.

    Provide a 'description' (a few sentences, 100-200 words) that advances the story and incorporates the AI concept for this turn, tailored to complexity level {turn_counter}.
    Provide a 'choices' array (2-4 distinct child-friendly options for the player, each a short phrase, related to AI exploration).
    Provide a 'score_change' (an an integer representing points gained).
    Provide a 'concept_explained' (a brief, child-friendly summary of the AI concept from this turn).

    The game should continue for a total of {TOTAL_GAME_TURNS} turns. Do not set 'gameOver' to true.
    If the game were to end, the description should clearly state the outcome (e.g., "You now understand how AI can learn from pictures! Game Over!").

    Remember to use simple analogies and avoid complex jargon. Focus on making the AI concept feel like a helpful or surprising part of their adventure.
    """

    scenario = await call_llm(game_prompt)

    if scenario:
        # Get the new score and concept from the LLM's response
        score_change = scenario.get('score_change', 0)
        concept_explained = scenario.get('concept_explained', '')
        correct_answer_feedback = scenario.get('correct_answer_feedback', '') # New field

        # Update the global player score
        player_score += score_change

        # Update the story description with the new information
        current_story_description = scenario['description']
        current_choices = scenario['choices']

        # Add feedback about the score and concept
        if score_change > 0:
            current_story_description += f"\n\n✨ You earned {score_change} points! ✨"
        elif correct_answer_feedback:
            # If the score is 0, add the learning feedback
            current_story_description += f"\n\n❌ Not quite! {correct_answer_feedback}"

        if concept_explained:
            # Add a specific marker around the concept explanation for UI styling
            current_story_description += f"\n\n<span class='ai-concept-highlight'>💡 AI Concept: {concept_explained}</span>"

        # Check if the game is over based on our turn counter
        if turn_counter >= TOTAL_GAME_TURNS:
            game_over_status = True
            info_message = f"Game Over! Your final score is {player_score}."
            current_choices = [] # Clear choices when game is over
    else:
        current_story_description = "An error occurred while continuing the adventure. Please try a different action or restart."
        current_choices = []
        game_over_status = True # End game on error
        error_message = "Error: Could not advance scenario."
    update_game_ui(is_loading=False)

# --- UI Generation Function ---
def update_game_ui(is_loading=False):
    """
    Updates the game's HTML user interface in the Google Colab output for the AI theme.
    Includes enhanced styling for a tech aesthetic, AI-themed icons,
    enhanced button effects, subtle animations, and visual highlighting for the AI concept.
    Also displays high scores loaded from Google Drive or in-memory storage.
    """
    global current_story_description, current_choices, game_over_status, loading_message, error_message, info_message, player_score, turn_counter, TOTAL_GAME_TURNS

    choices_html = ""
    input_area_html = ""
    high_scores_html = ""
    name_input_html = ""

    # Logic for when the game is NOT over
    if not game_over_status and not is_loading:
        for i, choice in enumerate(current_choices):
            # Add a number to each choice and make sure the onclick works with it
            escaped_choice_for_js = choice.replace("'", "\\'")
            choices_html += f"""
            <button
                class="choice-button bg-blue-600 hover:bg-blue-800 text-white font-bold py-2 px-4 rounded-full m-2 transition duration-300 ease-in-out transform hover:scale-105 shadow-lg"
                onclick="google.colab.kernel.invokeFunction('handle_player_action', ['{escaped_choice_for_js}'], {{}})"
            >
                {i+1}. {choice}
            </button>
            """
        input_area_html = f"""
        <div class="input-area">
            <input
                type="text"
                id="player-input"
                placeholder="Or type your own action here..."
                onkeydown="if(event.key === 'Enter') google.colab.kernel.invokeFunction('handle_player_action', [document.getElementById('player-input').value], {{}}); return event.key !== 'Enter';"
                class="rounded-lg border border-gray-600 focus:ring-blue-500 focus:border-blue-500"
            >
            <button
                class="go-button bg-green-600 hover:bg-green-800 text-white font-bold py-2 px-6 rounded-full transition duration-300 ease-in-out transform hover:scale-105 shadow-lg"
                onclick="google.colab.kernel.invokeFunction('handle_player_action', [document.getElementById('player-input').value], {{}})"
            >
                Go!
            </button>
        </div>
        """

    # Logic for when the game IS over
    if game_over_status and not is_loading:
        # Display the final score and prompt for name
        name_input_html = f"""
        <div class="input-area flex-col mt-4">
            <p class="text-xl font-bold">Your final score is: <span class="text-yellow-400">{player_score}</span></p>
            <p class="mt-2">Enter your name to save your score to the leaderboard!</p>
            <input type="text" id="player_name_input" placeholder="Your Name" class="rounded-lg border border-gray-600 focus:ring-blue-500 focus:border-blue-500 mb-2 mt-2">
            <button
                class="save-button bg-green-600 hover:bg-green-800 text-white font-bold py-2 px-6 rounded-full transition duration-300 ease-in-out transform hover:scale-105 shadow-lg"
                onclick="google.colab.kernel.invokeFunction('save_score_callback', [document.getElementById('player_name_input').value], {{}})"
            >
                Save Score
            </button>
        </div>
        """

        # Display the high scores table
        # This now reads from the Google Drive file or in-memory storage
        high_scores = get_high_scores()
        if high_scores:
            high_scores_html = f"""
            <div class="high-scores-box mt-4 p-4 rounded-lg bg-gray-700 shadow-inner">
                <h3 class="text-xl font-bold mb-2 text-blue-400">Top 5 AI Explorers</h3>
                <ul class="list-none mx-auto w-fit text-left">
            """
            for name, score in high_scores:
                high_scores_html += f"<li class='py-1 border-b border-gray-600'>{name}: <span class='font-bold text-yellow-400'>{score}</span> points</li>"
            high_scores_html += "</ul></div>"

    # Message area for errors or info
    message_area_class = "hidden"
    message_content = ""
    if error_message:
        message_area_class = "bg-red-700 text-white p-3 rounded-lg text-center"
        message_content = error_message
    elif info_message:
        message_area_class = "bg-blue-700 text-white p-3 rounded-lg text-center"
        message_content = info_message

    html_content = f"""
    <script src="https://cdn.tailwindcss.com"></script>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;700&display=swap" rel="stylesheet">
    <style>
        body {{
            font-family: 'Inter', sans-serif;
            /* Gradient background for tech feel */
            background: linear-gradient(135deg, #1f2937 0%, #374151 100%);
            color: #e5e7eb; /* Lighter text */
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            margin: 0;
            padding: 20px;
            box-sizing: border-box;
        }}
        .game-container {{
            background-color: #374151; /* Darker container */
            border-radius: 20px;
            padding: 30px;
            max-width: 800px;
            width: 100%;
            box-shadow: 0 10px 30px rgba(0, 0, 0, 0.7); /* Increased shadow */
            text-align: center;
            display: flex;
            flex-direction: column;
            gap: 20px;
            border: 2px solid #60a5fa; /* Border for definition */
        }}
        .title {{
            font-size: 2.5rem;
            font-weight: 700;
            color: #60a5fa; /* Brighter blue title */
            margin-bottom: 10px;
            display: flex;
            align-items: center;
            justify-content: center;
            text-shadow: 0 0 8px rgba(96, 165, 250, 0.8); /* Glow effect */
        }}
         .title .icon {{
            margin: 0 10px; /* Space around icons */
            font-size: 2.8rem; /* Slightly larger icons */
        }}
        .subtitle {{
            font-size: 1.5rem;
            color: #93c5fd; /* Lighter blue subtitle */
            margin-bottom: 20px;
        }}
        .story-box {{
            background-color: #4b5563; /* Even darker for story */
            border-radius: 15px;
            padding: 20px;
            min-height: 150px;
            display: flex;
            /* Changed to block to ensure proper text flow */
            display: block;
            font-size: 1.1rem;
            line-height: 1.6;
            text-align: left;
            overflow-y: auto;
            max-height: 300px; /* Limit height for scrollability */
            border: 1px solid #6b7280; /* Subtle border */
            box-shadow: inset 0 0 5px rgba(0, 0, 0, 0.3); /* Inner shadow */
            animation: text-fade-in 1s ease-out; /* Animation for new text */
            color: #e5e7eb; /* Ensure text color is light */
        }}
        /* Added style for highlighting AI concept */
        .ai-concept-highlight {{
            display: block; /* Make it a block element to have its own line */
            margin-top: 15px; /* Add space above */
            padding: 10px 15px; /* Add padding */
            background-color: rgba(96, 165, 250, 0.2); /* Light blue background */
            border-left: 5px solid #60a5fa; /* Blue left border */
            border-radius: 5px;
            font-weight: bold;
            color: #93c5fd; /* Lighter blue text */
            animation: concept-pop-in 0.6s ease-out; /* Animation for concept highlight */
        }}
         @keyframes concept-pop-in {{
            0% {{ opacity: 0; transform: translateY(10px); }}
            100% {{ opacity: 1; transform: translateY(0); }}
        }}

        .choices-container {{
            display: flex;
            flex-wrap: wrap;
            justify-content: center;
            gap: 10px;
            margin-top: 20px;
        }}
        .choice-button, .go-button, .save-button, .start-button {{
             /* Existing styles... */
             transition: all 0.3s ease-in-out;
        }}
        .choice-button:hover, .go-button:hover, .save-button:hover, .start-button:hover {{
             transform: scale(1.08); /* Slightly more pronounced hover effect */
             box-shadow: 0 5px 15px rgba(0, 0, 0, 0.3); /* Add shadow on hover */
        }}
         @keyframes pulse {{
            0% {{ box-shadow: 0 0 0 0 rgba(96, 165, 250, 0.7); }}
            70% {{ box-shadow: 0 0 0 15px rgba(96, 165, 250, 0); }}
            100% {{ box-shadow: 0 0 0 0 rgba(96, 165, 250, 0); }}
        }}
        .start-button {{
            animation: pulse 2s infinite; /* Add pulse animation to start button */
        }}

         @keyframes text-fade-in {{
            0% {{ opacity: 0; }}
            100% {{ opacity: 1; }}
        }}

        .input-area {{
            display: flex;
            flex-direction: column;
            gap: 10px;
            margin-top: 20px;
            align-items: center;
        }}
        .input-area input {{
            width: calc(100% - 20px);
            max-width: 400px;
            padding: 12px 15px;
            border-radius: 10px;
            border: 1px solid #6b7280;
            background-color: #2d3748; /* Slightly lighter input background */
            color: #e5e7eb;
            font-size: 1rem;
        }}
        .input-area input:focus {{
            outline: none;
            border-color: #60a5fa;
            box-shadow: 0 0 0 3px rgba(96, 165, 250, 0.5);
        }}
        .control-buttons {{
            display: flex;
            justify-content: center;
            gap: 15px;
            margin-top: 20px;
        }}
        .message-box {{
            margin-top: 20px;
            padding: 15px;
            border-radius: 10px;
            font-weight: bold;
        }}
        .hidden {{
            display: none;
        }}
        /* Responsive adjustments */
        @media (max-width: 600px) {{
            .game-container {{
                padding: 20px;
            }}
            .title {{
                font-size: 2rem;
            }}
             .title .icon {{
                font-size: 2.2rem; /* Adjust icon size */
            }}
            .subtitle {{
                font-size: 1.2rem;
            }}
            .story-box {{
                font-size: 1rem;
            }}
        }}
    </style>

    <div class="game-container">
        <div class="title">
            <span class="icon">🤖</span> AI Explorer! <span class="icon">🧠</span>
        </div>
        <div class="subtitle">
            An Adventure into the World of Smart Machines
        </div>
        <p class="text-lg font-bold text-yellow-400">Score: {player_score} | Turn: {turn_counter} / {TOTAL_GAME_TURNS}</p>
        <div class="story-box">
            {current_story_description}
            {is_loading and f'<div class="text-gray-400 mt-4">{loading_message}</div>' or ''}
        </div>

        <div class="choices-container">
            {choices_html}
            {name_input_html}
        </div>

        {input_area_html}

        <div class="control-buttons">
            <button
                class="start-button bg-purple-700 hover:bg-purple-900 text-white font-bold py-2 px-6 rounded-full transition duration-300 ease-in-out transform hover:scale-105 shadow-lg"
                onclick="google.colab.kernel.invokeFunction('start_new_game', [], {{}})"
            >
                Start New AI Adventure
            </button>
        </div>
        {high_scores_html}
        <div class="{message_area_class}">
            {message_content}
        </div>
    </div>
    """
    # Clear the cell and display new HTML content
    display.clear_output(wait=True)
    display.display(display.HTML(html_content))

def save_score_callback(player_name):
    """
    A synchronous wrapper to save the high score from the UI.
    Saves the player's score to the high scores CSV file in Google Drive or in-memory storage.
    """
    global player_score, info_message, error_message
    if player_name and player_name.strip():
        try:
            # Load existing scores
            existing_scores = get_high_scores() # Use the alias which points to Drive or in-memory
            # Add the new score (rounded to nearest integer for saving)
            existing_scores.append((player_name, round(player_score)))
            # Sort by score descending
            existing_scores.sort(key=lambda item: item[1], reverse=True)
            # Save the updated list (only keeping top 5 for the leaderbard display is handled in get_high_scores)
            save_high_scores_to_drive(existing_scores) # Use the alias which points to Drive or in-memory
            info_message = f"Score for {player_name} ({round(player_score)}) has been saved to the leaderboard!"
            error_message = "" # Clear any previous errors
        except Exception as e:
            error_message = f"Error saving score: {e}"
            info_message = ""
            print(f"Error in save_score_callback: {e}")
    else:
        error_message = "Please enter a name to save your score."
        info_message = "" # Clear any previous info
    update_game_ui(is_loading=False) # Update UI to show confirmation or error

# Main Execution

# Define synchronous wrappers to call the async functions
def sync_start_new_game():
    asyncio.run(start_new_game())

def sync_handle_player_action(player_action):
    asyncio.run(handle_player_action(player_action))

# Register Python functions as callbacks callable from JavaScript
# These must be called after the functions are defined.
output.register_callback('start_new_game', sync_start_new_game)
output.register_callback('handle_player_action', sync_handle_player_action)
output.register_callback('save_score_callback', save_score_callback)

# Initial display of the game UI
update_game_ui()